# ***Consumer Confidence 2018, 2020 and 2022***
---
This notebook contains code to convert the data from the 2018, 2020 and 2022 National Consumer Confidence Survey (ENCO), for subsequent comparison with the data from the National Household Income and Expenditure Survey (ENIGH) for the same year.


First, the libraries necessary for the process will be loaded.

In [2]:
import requests
import zipfile
import os
import pandas as pd
import numpy as np
from io import BytesIO

Using the following functions, the .csv files for the years 2018, 2020, and 2022 are downloaded and unzipped.

In [3]:
# Years and exceptions for files with different names
years = {
    2018: {
        "pattern": "conjunto_de_datos_enco_2018_{month}_csv",
        "exceptions": {
            "01": "enco_enero_2018_csv",
            "02": "enco_febrero_2018_csv",
            "03": "enco_marzo_2018_csv",
            "04": "enco_abril_2018_csv",
            "05": "enco_mayo_2018_csv",
            "06": "conjunto_de_datos_enco0618_csv",
            "07": "conjunto_de_datos_enco0718_csv"
        }
    },
    2020: {
        "pattern": "conjunto_de_datos_enco_2020_{month}_csv",
        "exceptions": {
            "04": None, "05": None, "06": None, "07": None  # Missing months
        }
    },
    2022: {
        "pattern": "conjunto_de_datos_enco_2022_{month}_csv"
    }
}
# Base URL for INEGI
base_url = "https://www.inegi.org.mx/contenidos/programas/enco/datosabiertos/{year}/{filename}.zip"

# Function to generate URLs for each year
def construir_url(year, month, info):
    if "exceptions" in info and month in info["exceptions"]:
        filename = info["exceptions"][month]
        if filename is None:
            return None  # Month with no file available
    else:
        filename = info["pattern"].format(month=month)
    return base_url.format(year=year, filename=filename)

# Function to download and extract zip files
def descargar_y_extraer_zip(url, extract_path):
    if not url:
        return

    response = requests.get(url)
    if response.status_code == 200:
        try:
            with zipfile.ZipFile(BytesIO(response.content)) as z:
                z.extractall(extract_path)
        except zipfile.BadZipFile:
            print(f"El archivo descargado de {url} no es un archivo zip válido")
    else:
        print(f"Error al descargar {url}, código de estado: {response.status_code}")

# Download the files for each year
for year, info in years.items():
    extract_path = f'/content/enco_{year}'
    os.makedirs(extract_path, exist_ok=True)

    for month in [str(i).zfill(2) for i in range(1, 13)]:
        url = construir_url(year, month, info)
        descargar_y_extraer_zip(url, extract_path)

Now, let's upload, filter, and merge ENCO survey datasets from the years 2018, 2020, and 2022 over the course of different months. Specific columns from three types of datasets (cs, viv, cb) are read, combined based on common columns, and the results are concatenated into a final ordered DataFrame. Ultimately, the code saves this final processed dataset in a CSV file for later analysis.

In [4]:
# Define the common and specific columns for each dataset
columnas_comunes = ['fol', 'ent', 'con', 'v_sel', 'n_hog', 'h_mud']

# Specific columns for each file
viv_especificas = ['mpio', 'ageb', 'fch_def']
cs_especificas = ['i_per', 'ing']
cb_especificas = [f'p{i}' for i in range(1, 16)]  # 'p1' a 'p15' para 'cb'

# Define the columns to be used in each dataset
viv_cols = columnas_comunes + viv_especificas
cs_cols = columnas_comunes + cs_especificas
cb_cols = columnas_comunes + cb_especificas

# Function to select relevant columns after normalizing them
def seleccionar_columnas(df, columnas_relevantes):
    if not df.empty:
        # Normalizes the column names of the DataFrame to lowercase if it is not empty
        df.columns = df.columns.str.lower()
        # Select only the columns that exist in the DataFrame
        columnas_existentes = [col for col in columnas_relevantes if col in df.columns]
        return df[columnas_existentes]
    return pd.DataFrame()  # If the DataFrame is empty, return an empty one

# Define the years to process
anios = [2018, 2020, 2022]

# Function to load data by year, month and set type
def cargar_datos(anio, mes, tipo):
    if anio == 2018:
        if mes <= 6:  # January to June
            file_name = f'enco{tipo}_0{mes}18.csv'
            folder_path = f'/content/enco_2018/{tipo}_enco0{mes}18/conjunto_de_datos'
        elif mes == 7:  # July
            file_name = f'conjunto_de_datos_enco{tipo}_0718.csv'
            folder_path = f'/content/enco_2018/{tipo}_enco0718/conjunto_de_datos'
        else:  # August to December
            file_name = f'conjunto_de_datos_{tipo}_enco_2018_{str(mes).zfill(2)}.csv'
            folder_path = f'/content/enco_2018/conjunto_de_datos_{tipo}_enco_2018_{str(mes).zfill(2)}/conjunto_de_datos'
    else:
        file_name = f'conjunto_de_datos_{tipo}_enco_{anio}_{str(mes).zfill(2)}.csv'
        folder_path = f'/content/enco_{anio}/conjunto_de_datos_{tipo}_enco_{anio}_{str(mes).zfill(2)}/conjunto_de_datos'

    # Normalize File Name
    file_path = os.path.join(folder_path, file_name)

    # Check if the file doesn't exist
    if not os.path.exists(file_path):
        file_name = file_name.replace('.csv', '.CSV')
        file_path = os.path.join(folder_path, file_name)

    # Check if the file exist
    if os.path.exists(file_path):
       return pd.read_csv(file_path)

    # If it is not found, it returns an empty DataFrame
    return pd.DataFrame()

# Iterate through the years and months to load and filter the data
df_final = pd.DataFrame()

for anio in anios:
    cs_enco_filtrado = [seleccionar_columnas(cargar_datos(anio, i, 'cs'), cs_cols) for i in range(1, 13)]
    viv_enco_filtrado = [seleccionar_columnas(cargar_datos(anio, i, 'viv'), viv_cols) for i in range(1, 13)]
    cb_enco_filtrado = [seleccionar_columnas(cargar_datos(anio, i, 'cb'), cb_cols) for i in range(1, 13)]

    for cs_df, viv_df, cb_df in zip(cs_enco_filtrado, viv_enco_filtrado, cb_enco_filtrado):
        # Only merge if the DataFrames are not empty
        if not cs_df.empty and not viv_df.empty and not cb_df.empty:
            temp = pd.merge(cs_df, viv_df, on=columnas_comunes, how='inner')
            temp = pd.merge(temp, cb_df, on=columnas_comunes, how='inner')
            df_final = pd.concat([df_final, temp], ignore_index=True)

# Mostrar las primeras filas del DataFrame final
df_final

,fol,ent,con,v_sel,n_hog,h_mud,i_per,ing,mpio,ageb,...,p6,p7,p8,p9,p10,p11,p12,p13,p14,p15
0,11B167,1,40006,3,1,0,1.0,1300.0,5,025-1,...,2,3,3,2,2,3,6,3,3,1
1,11B167,1,40006,3,1,0,NaN,NaN,5,025-1,...,2,3,3,2,2,3,6,3,3,1
2,11B167,1,40006,3,1,0,NaN,NaN,5,025-1,...,2,3,3,2,2,3,6,3,3,1
3,11B167,1,40006,3,1,0,NaN,NaN,5,025-1,...,2,3,3,2,2,3,6,3,3,1
4,11B167,1,40006,3,1,0,1.0,1100.0,5,025-1,...,2,3,3,2,2,3,6,3,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
217135,12B212,32,40385,4,1,0,,NaN,56,037-9,...,3,3,3,2,2,2,6,3,3,3
217136,12B212,32,40385,4,1,0,,NaN,56,037-9,...,3,3,3,2,2,2,6,3,3,3
217137,12B212,32,40385,4,1,0,2,4000.0,56,037-9,...,3,3,3,2,2,2,6,3,3,3
217138,12B212,32,40385,3,1,0,,NaN,56,037-9,...,2,1,1,1,1,2,2,2,3,2


We'll take a look at the descriptive statistics of the data, as well as see the data lost by each column.

In [5]:
# Getting basic statistics from the final DataFrame
print("Basic statistics of the final DataFrame:")
df_final.describe(include='all')

Basic statistics of the final DataFrame:


,fol,ent,con,v_sel,n_hog,h_mud,i_per,ing,mpio,ageb,...,p6,p7,p8,p9,p10,p11,p12,p13,p14,p15
count,217140,217140.000000,217140.000000,217140.000000,217140.0,217140.000000,172199,91928.000000,217140.000000,217140,...,217140.000000,217140.000000,217140.000000,217140.000000,217140.000000,217140.000000,217140.000000,217140.000000,217140.000000,217140.000000
unique,219,NaN,NaN,NaN,NaN,NaN,11,NaN,NaN,3235,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,11A201,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,025-1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,1818,NaN,NaN,NaN,NaN,NaN,79075,NaN,NaN,393,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,15.686824,40295.016086,2.503007,1.0,0.040743,NaN,384434.426236,41.076932,NaN,...,3.226766,2.355047,2.554426,1.749572,1.764728,3.062605,4.936239,3.249940,2.781721,2.688169
std,NaN,7.837322,491.438315,1.115752,0.0,0.212124,NaN,484181.879242,52.167613,NaN,...,1.038131,0.725403,0.613975,0.463439,0.565478,0.849968,1.143216,1.070032,0.563126,0.659151
min,NaN,1.000000,22251.000000,1.000000,1.0,0.000000,NaN,25.000000,1.000000,NaN,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,NaN,9.000000,40151.000000,2.000000,1.0,0.000000,NaN,1500.000000,6.000000,NaN,...,2.000000,2.000000,2.000000,1.000000,1.000000,3.000000,4.000000,3.000000,3.000000,3.000000
50%,NaN,15.000000,40271.000000,3.000000,1.0,0.000000,NaN,5000.000000,26.000000,NaN,...,3.000000,3.000000,3.000000,2.000000,2.000000,3.000000,5.000000,3.000000,3.000000,3.000000
75%,NaN,20.000000,40391.000000,3.000000,1.0,0.000000,NaN,999999.000000,50.000000,NaN,...,4.000000,3.000000,3.000000,2.000000,2.000000,4.000000,6.000000,4.000000,3.000000,3.000000


In [6]:
# Check for missing data
print("Missing data in the final DataFrame:")
missing_data = df_final.isnull().sum()
missing_data

Missing data in the final DataFrame:


,0
fol,0
ent,0
con,0
v_sel,0
n_hog,0
h_mud,0
i_per,44941
ing,125212
mpio,0
ageb,0


In [7]:
# Define the values that represent lost data
valores_perdidos = [9999, 99999, 999999]

# Function to replace lost values with NaN
def reemplazar_valores_perdidos(df, valores):
    return df.replace(valores, np.nan)

# Data Quality Analysis Function
def analizar_calidad_datos(df):
    # Replace lost values with NaN
    df_limpio = reemplazar_valores_perdidos(df, valores_perdidos)
    # Calculate the percentage of missing values per column
    porcentaje_perdidos = df_limpio.isnull().mean() * 100
    # Show columns that have missing values
    print("Porcentaje de valores perdidos por columna:")
    print(porcentaje_perdidos[porcentaje_perdidos > 0])
    return df_limpio

# Analyze data quality
df_final_limpio = analizar_calidad_datos(df_final)

# We keep the DataFrame clean
df_final_limpio.to_csv('/content/datos_limpios.csv', index=False)

Porcentaje de valores perdidos por columna:
i_per    20.696785
ing      74.693746
dtype: float64


In [8]:
limpio = reemplazar_valores_perdidos(df_final, valores_perdidos)
print("Missing data in the final DataFrame:")
missing_data = limpio.isnull().sum()
missing_data

Missing data in the final DataFrame:


,0
fol,0
ent,0
con,0
v_sel,0
n_hog,0
h_mud,0
i_per,44941
ing,162190
mpio,0
ageb,0
